# import all required packages

In [1]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import random
import seaborn as sns
from shapely.ops import linemerge
from shapely.wkt import loads
from shapely.geometry import LineString
from shapely.geometry import Point, Polygon
from scipy.spatial import Delaunay

# Hydrant dataframe

In [2]:
df_point = gpd.read_file('hydrant_location.shp')
df_point['id'] = df_point.reset_index().index
df_point

,geometry,id
0,POINT (-74.00698 40.74748),0
1,POINT (-74.00722 40.74722),1
2,POINT (-74.00721 40.74697),2
3,POINT (-74.00722 40.74722),3
4,POINT (-74.00728 40.74667),4
...,...,...
2295,POINT (-73.97976 40.71559),2295
2296,POINT (-73.97987 40.71361),2296
2297,POINT (-73.98387 40.72077),2297
2298,POINT (-73.97780 40.71833),2298


# Road dataframe

In [3]:
df_road = gpd.read_file('road_location/road_location.shp')
df_road.drop('road_id', axis=1, inplace=True)
df_road

,id,geometry
0,1,"LINESTRING (-74.01301 40.70215, -74.01167 40.7..."
1,1,"LINESTRING (-74.01171 40.70247, -74.00999 40.7..."
2,1,"LINESTRING (-74.00577 40.70566, -74.00396 40.7..."
3,1,"LINESTRING (-74.00385 40.70651, -74.00142 40.7..."
4,2,"LINESTRING (-74.01270 40.70275, -74.01180 40.7..."
...,...,...
331,248,"LINESTRING (-73.99741 40.72164, -73.99389 40.7..."
332,249,"LINESTRING (-74.00203 40.71159, -74.00252 40.7..."
333,250,"LINESTRING (-74.00296 40.71334, -74.00150 40.7..."
334,251,"LINESTRING (-74.00799 40.70899, -74.00720 40.7..."


# Find closest road & close roads

In [4]:
def find_close_road(point, roads_df):
    close_road = None
    min_distance = float('inf')

    for idx, road in roads_df.iterrows():
        distance = point.distance(road['geometry'])
        if distance < min_distance:
            min_distance = distance
            close_road = road['id']
    return close_road

In [5]:
df_point['close_road'] = df_point.apply(lambda row: find_close_road(row['geometry'], df_road), axis=1)

In [6]:
def find_closest_road(point, roads_df, min_distance):
    closest_road_id = []

    for idx, road in roads_df.iterrows():
        distance = point.distance(road['geometry'])
        if distance < min_distance:
            closest_road_id.append(road['id'])

    return closest_road_id

In [7]:
df_point['closest_road_id'] = df_point.apply(lambda row: find_closest_road(row['geometry'], df_road, 0.0004), axis=1)

In [8]:
df_point

,geometry,id,close_road,closest_road_id
0,POINT (-74.00698 40.74748),0,119,"[119, 170]"
1,POINT (-74.00722 40.74722),1,119,"[119, 120, 170]"
2,POINT (-74.00721 40.74697),2,120,"[120, 170]"
3,POINT (-74.00722 40.74722),3,119,"[119, 120, 170]"
4,POINT (-74.00728 40.74667),4,120,"[120, 121, 170]"
...,...,...,...,...
2295,POINT (-73.97976 40.71559),2295,234,"[33, 234]"
2296,POINT (-73.97987 40.71361),2296,30,"[30, 35, 235]"
2297,POINT (-73.98387 40.72077),2297,221,[221]
2298,POINT (-73.97780 40.71833),2298,199,"[199, 235]"


# Find block id for each hydrant

In [9]:
def blockInfo(shapefile):
    gdf = gpd.read_file(shapefile)
    block_info_list = []
    for idx, row in gdf.iterrows():
        block_id = row['id']
        polygon = row['geometry']
        corners = list(polygon.exterior.coords)
        block_info = {
            'blockID': block_id,
            'corners': corners
        }
        block_info_list.append(block_info)
    return block_info_list

In [10]:
def blockIdentify(hydrantCenter, blocks):
    hydrantPoint = Point(hydrantCenter)
    for block in blocks:
        blockCoords = block['corners']
        blockPolygon = Polygon(blockCoords)
        if hydrantPoint.within(blockPolygon):
            hydrantBlockID = block['blockID']
            return hydrantBlockID
    return None

In [11]:
blocks = blockInfo("block_location/blocksagain.shp")
df_point['block_id'] = df_point.apply(lambda row: blockIdentify(row['geometry'], blocks), axis=1)

In [12]:
# check if any NaN in block_id
block_nan_count = df_point['block_id'].isna().sum()
block_nan_count

131

In [13]:
df_point

,geometry,id,close_road,closest_road_id,block_id
0,POINT (-74.00698 40.74748),0,119,"[119, 170]",1.0
1,POINT (-74.00722 40.74722),1,119,"[119, 120, 170]",2.0
2,POINT (-74.00721 40.74697),2,120,"[120, 170]",2.0
3,POINT (-74.00722 40.74722),3,119,"[119, 120, 170]",2.0
4,POINT (-74.00728 40.74667),4,120,"[120, 121, 170]",3.0
...,...,...,...,...,...
2295,POINT (-73.97976 40.71559),2295,234,"[33, 234]",505.0
2296,POINT (-73.97987 40.71361),2296,30,"[30, 35, 235]",901.0
2297,POINT (-73.98387 40.72077),2297,221,[221],437.0
2298,POINT (-73.97780 40.71833),2298,199,"[199, 235]",444.0


# Convert all hydrants and edges into Class

In [14]:
class Hydrant:

    def __init__(self, id, x, y, block_id, closest_road_id, close_road): # constructor

        # From shapefile
        self.pointID = id
        self.x = x
        self.y = y
        self.blockID = block_id
        self.roadID = closest_road_id
        self.closeRoadID = close_road

        # Will initialize once into network generation
        self.branchID = 0
        self.neighbor = [] # List to store all the neighbors of this hydrant
        self.edge = [] # Dict to store edge information

        self.elevation = 0

In [60]:
class Edge:

    def __init__(self, id, p1, p2, length): # constructor

        self.edgeID = id
        self.p1 = p1 # id
        self.p2 = p2 # id
        self.length = length
        self.angle_difference = 0
        self.slope_differece = 0
        self.cost = 0

In [16]:
# Create an empty list to store the 'hydrant' instances
hydrant_instances = []

# Iterate through each row in the DataFrame and create 'hydrant' instances
for index, row in df_point.iterrows():
    # Get the Point object from the 'geometry' column
    point = row['geometry']

    # Extract 'x' and 'y' coordinates from the Point object
    x, y = point.x, point.y

    # Create 'hydrant' instance and add it to the list
    hydrant_obj = Hydrant(row['id'], x, y, row['block_id'], row['closest_road_id'], row['close_road'])
    hydrant_instances.append(hydrant_obj)

# Distance calculation method

In [17]:
def distance(p1, p2):
    p1Lat, p1Long = p1[0], p1[1]
    p2Lat, p2Long = p2[0], p2[1]
    R = 6371000  # Earth's radius in meters
    lat1_rad = math.radians(p1Lat)
    lon1_rad = math.radians(p1Long)
    lat2_rad = math.radians(p2Lat)
    lon2_rad = math.radians(p2Long)

    delta_lat = lat2_rad - lat1_rad
    delta_lon = lon2_rad - lon1_rad

    a = math.sin(delta_lat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(delta_lon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance

# Calculate Elavation of each hydrant

In [18]:
elevation_shapefile_path = 'hydrant_locations_with_rasterdata/hydrant_locations_with_rasterdata.shp'
first_hydrant_raster_gdf = gpd.read_file(elevation_shapefile_path)

second_elevation_shapefilepath = 'second_hydrants_with_raster/second_hydrants_with_raster.shp'
second_hydrant_raster_gdf = gpd.read_file(second_elevation_shapefilepath)

In [19]:
def create_hydrants_elev_dict(first_hydrant_raster_gdf, second_hydrant_raster_gdf):

  hydrants_to_elev = {}

  for index, row in first_hydrant_raster_gdf.iterrows():
    hydrant_point = row['geometry']
    if pd.isna(row['SAMPLE_1']): #if the value is not in the first shapefile DEM, check second DEM file
        corresponding_elev = second_hydrant_raster_gdf.loc[index, 'ELEV1'] #call the corresponding elevation from second DEM file
        hydrants_to_elev[(hydrant_point.x, hydrant_point.y)] = corresponding_elev #update the dictionary value
    else:
        hydrants_to_elev[(hydrant_point.x, hydrant_point.y)] = row['SAMPLE_1']
  return hydrants_to_elev

In [20]:
elevation_data = create_hydrants_elev_dict(first_hydrant_raster_gdf, second_hydrant_raster_gdf)

In [21]:
# Create a new column 'elevation' in the df_point DataFrame and set it to None initially
df_point['elevation'] = None

# Iterate through the df_point DataFrame and update the 'elevation' column with the corresponding elevation from the dictionary
for index, row in df_point.iterrows():
    geometry = row['geometry']
    elevation = elevation_data.get((geometry.x, geometry.y), None)
    df_point.at[index, 'elevation'] = elevation * 0.3048
    hydrant_instances[index].elevation = elevation

In [22]:
df_point

,geometry,id,close_road,closest_road_id,block_id,elevation
0,POINT (-74.00698 40.74748),0,119,"[119, 170]",1.0,1.606296
1,POINT (-74.00722 40.74722),1,119,"[119, 120, 170]",2.0,2.084832
2,POINT (-74.00721 40.74697),2,120,"[120, 170]",2.0,2.465832
3,POINT (-74.00722 40.74722),3,119,"[119, 120, 170]",2.0,2.060448
4,POINT (-74.00728 40.74667),4,120,"[120, 121, 170]",3.0,2.267712
...,...,...,...,...,...,...
2295,POINT (-73.97976 40.71559),2295,234,"[33, 234]",505.0,3.62712
2296,POINT (-73.97987 40.71361),2296,30,"[30, 35, 235]",901.0,6.5532
2297,POINT (-73.98387 40.72077),2297,221,[221],437.0,7.303008
2298,POINT (-73.97780 40.71833),2298,199,"[199, 235]",444.0,2.77368


# Delaunay Triangulation

In [23]:
# Extract x and y coordinates from hydrant instances
hydrant_points = [(hydrant.x, hydrant.y) for hydrant in hydrant_instances]

# Convert the list of points to a numpy array
points = np.array(hydrant_points)

# Perform Delaunay triangulation
triangulation = Delaunay(points)

# Initial set with all edges

In [24]:
filtered_edges_road_cross = set()
edge_set_pro = []

In [25]:
max_edge_length = 166.804848

# track edgeID
edge_id = 0

# Loop through each triangle (simplex)
for simplex in triangulation.simplices:
    
    # Get the indices of the points forming the edges of the triangle
    edge1 = frozenset([simplex[0], simplex[1]])
    edge2 = frozenset([simplex[1], simplex[2]])
    edge3 = frozenset([simplex[0], simplex[2]])

    # Get the actual points from the 'points' list using the indices
    point1, point2, point3 = points[simplex[0]], points[simplex[1]], points[simplex[2]]
    
    # Get the actual points from the 'hydrant_instances' list using the indices
    hydrant1, hydrant2, hydrant3 = hydrant_instances[simplex[0]], hydrant_instances[simplex[1]], hydrant_instances[simplex[2]]
    
    # Calculate the edge lengths
    edge1_length = distance(point1, point2)
    edge2_length = distance(point2, point3)
    edge3_length = distance(point1, point3)

    if edge1_length <= max_edge_length:
        edge_obj = Edge(edge_id, simplex[0], simplex[1], edge1_length)
        edge_set_pro.append(edge_obj)
        edge_id += 1
        # Include the edge if elevation from hydrant1 to hydrant2 is greater than the threshold
        if hydrant1.elevation - hydrant2.elevation >= 0:
            hydrant1.neighbor.append(hydrant2)
            hydrant1.edge.append(edge_obj)
        else:
            hydrant2.neighbor.append(hydrant1)
            hydrant2.edge.append(edge_obj)
        
    if edge2_length <= max_edge_length:
        edge_obj = Edge(edge_id, simplex[1], simplex[2], edge2_length)
        edge_set_pro.append(edge_obj)
        edge_id += 1
        # Include the edge if elevation from hydrant1 to hydrant2 is greater than the threshold
        if hydrant2.elevation - hydrant3.elevation >= 0:
            hydrant2.neighbor.append(hydrant3)
            hydrant2.edge.append(edge_obj)
        else:
            hydrant3.neighbor.append(hydrant2)
            hydrant3.edge.append(edge_obj)
    
    if edge3_length <= max_edge_length:
        edge_obj = Edge(edge_id, simplex[0], simplex[2], edge3_length)
        edge_set_pro.append(edge_obj)
        edge_id += 1
        # Include the edge if elevation from hydrant1 to hydrant2 is greater than the threshold
        if hydrant1.elevation - hydrant3.elevation >= 0:
            hydrant1.neighbor.append(hydrant3)
            hydrant1.edge.append(edge_obj)
        else:
            hydrant3.neighbor.append(hydrant1)
            hydrant3.edge.append(edge_obj)

In [26]:
len(edge_set_pro)

13454

# Calculate each road's angle to NORTH

In [27]:
# Calculate the angle of each road relative to North
def calculate_angle_to_north(geometry):
    # Get the start and end points of the LineString
    start_point, end_point = geometry.coords[0], geometry.coords[-1]

    # Calculate the difference in longitude and latitude
    delta_lon = end_point[0] - start_point[0]
    delta_lat = end_point[1] - start_point[1]

    # Calculate the angle using atan2 and convert it to degrees
    angle_to_north = math.degrees(math.atan2(delta_lat, delta_lon))

    return angle_to_north

In [28]:
# Apply the calculate_angle_to_north function to the 'geometry' column
df_road['angle_to_north'] = df_road['geometry'].apply(calculate_angle_to_north)

In [29]:
df_road

,id,geometry,angle_to_north
0,1,"LINESTRING (-74.01301 40.70215, -74.01167 40.7...",13.912495
1,1,"LINESTRING (-74.01171 40.70247, -74.00999 40.7...",28.244251
2,1,"LINESTRING (-74.00577 40.70566, -74.00396 40.7...",29.744881
3,1,"LINESTRING (-74.00385 40.70651, -74.00142 40.7...",24.921771
4,2,"LINESTRING (-74.01270 40.70275, -74.01180 40.7...",15.296547
...,...,...,...
331,248,"LINESTRING (-73.99741 40.72164, -73.99389 40.7...",-20.160698
332,249,"LINESTRING (-74.00203 40.71159, -74.00252 40.7...",117.395367
333,250,"LINESTRING (-74.00296 40.71334, -74.00150 40.7...",15.592811
334,251,"LINESTRING (-74.00799 40.70899, -74.00720 40.7...",45.000000


# Create edge dataframe

In [30]:
def edge_attributes_to_list(edges):
    edge_list = []
    for edge in edges:
        edge_dict = {
            'edgeID': edge.edgeID,
            'p1': edge.p1,
            'p2': edge.p2,
            'length': edge.length,
        }
        edge_list.append(edge_dict)
    return edge_list

# Call the function to convert the Edge objects to a list of dictionaries
edge_list = edge_attributes_to_list(edge_set_pro)

# Create a DataFrame from the list of dictionaries
edge_df = pd.DataFrame(edge_list)

# Display the DataFrame containing edge information
edge_df

,edgeID,p1,p2,length
0,0,293,312,73.819498
1,1,312,304,28.143167
2,2,293,304,45.696590
3,3,312,307,80.053064
4,4,307,304,61.087238
...,...,...,...,...
13449,13449,436,441,60.763325
13450,13450,445,441,30.563361
13451,13451,445,435,52.070980
13452,13452,435,436,9.040822


In [31]:
edge_df = edge_df.drop_duplicates(subset=['p1', 'p2', 'length'])

In [32]:
edge_df

,edgeID,p1,p2,length
0,0,293,312,73.819498
1,1,312,304,28.143167
2,2,293,304,45.696590
3,3,312,307,80.053064
4,4,307,304,61.087238
...,...,...,...,...
13442,13442,883,912,46.474117
13445,13445,883,911,36.598551
13446,13446,911,912,17.600841
13448,13448,445,436,48.371650


In [33]:
merged_df = pd.merge(edge_df, df_point, left_on='p1', right_on='id', how='left')

In [34]:
merged_df['geometry_p1'] = merged_df['geometry']
merged_df

,edgeID,p1,p2,length,geometry,id,close_road,closest_road_id,block_id,elevation,geometry_p1
0,0,293,312,73.819498,POINT (-74.01267 40.71804),293,174,"[88, 174]",NaN,3.023616,POINT (-74.01267 40.71804)
1,1,312,304,28.143167,POINT (-74.01311 40.71625),312,174,[174],NaN,3.194304,POINT (-74.01311 40.71625)
2,2,293,304,45.696590,POINT (-74.01267 40.71804),293,174,"[88, 174]",NaN,3.023616,POINT (-74.01267 40.71804)
3,3,312,307,80.053064,POINT (-74.01311 40.71625),312,174,[174],NaN,3.194304,POINT (-74.01311 40.71625)
4,4,307,304,61.087238,POINT (-74.01239 40.71655),307,79,[79],622.0,2.740152,POINT (-74.01239 40.71655)
...,...,...,...,...,...,...,...,...,...,...,...
8981,13442,883,912,46.474117,POINT (-74.00483 40.71329),883,80,[80],706.0,12.429744,POINT (-74.00483 40.71329)
8982,13445,883,911,36.598551,POINT (-74.00483 40.71329),883,80,[80],706.0,12.429744,POINT (-74.00483 40.71329)
8983,13446,911,912,17.600841,POINT (-74.00464 40.71233),911,22,"[18, 22]",NaN,12.807696,POINT (-74.00464 40.71233)
8984,13448,445,436,48.371650,POINT (-74.01158 40.70319),445,2,"[2, 2, 10, 4]",735.0,1.73736,POINT (-74.01158 40.70319)


In [35]:
merged_df = pd.merge(merged_df, df_point, left_on='p2', right_on='id', how='left')

In [36]:
merged_df

,edgeID,p1,p2,length,geometry_x,id_x,close_road_x,closest_road_id_x,block_id_x,elevation_x,geometry_p1,geometry_y,id_y,close_road_y,closest_road_id_y,block_id_y,elevation_y
0,0,293,312,73.819498,POINT (-74.01267 40.71804),293,174,"[88, 174]",NaN,3.023616,POINT (-74.01267 40.71804),POINT (-74.01311 40.71625),312,174,[174],NaN,3.194304
1,1,312,304,28.143167,POINT (-74.01311 40.71625),312,174,[174],NaN,3.194304,POINT (-74.01311 40.71625),POINT (-74.01294 40.71692),304,174,"[79, 80, 174]",NaN,3.255264
2,2,293,304,45.696590,POINT (-74.01267 40.71804),293,174,"[88, 174]",NaN,3.023616,POINT (-74.01267 40.71804),POINT (-74.01294 40.71692),304,174,"[79, 80, 174]",NaN,3.255264
3,3,312,307,80.053064,POINT (-74.01311 40.71625),312,174,[174],NaN,3.194304,POINT (-74.01311 40.71625),POINT (-74.01239 40.71655),307,79,[79],622.0,2.740152
4,4,307,304,61.087238,POINT (-74.01239 40.71655),307,79,[79],622.0,2.740152,POINT (-74.01239 40.71655),POINT (-74.01294 40.71692),304,174,"[79, 80, 174]",NaN,3.255264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8981,13442,883,912,46.474117,POINT (-74.00483 40.71329),883,80,[80],706.0,12.429744,POINT (-74.00483 40.71329),POINT (-74.00448 40.71246),912,22,"[18, 22]",NaN,11.97864
8982,13445,883,911,36.598551,POINT (-74.00483 40.71329),883,80,[80],706.0,12.429744,POINT (-74.00483 40.71329),POINT (-74.00464 40.71233),911,22,"[18, 22]",NaN,12.807696
8983,13446,911,912,17.600841,POINT (-74.00464 40.71233),911,22,"[18, 22]",NaN,12.807696,POINT (-74.00464 40.71233),POINT (-74.00448 40.71246),912,22,"[18, 22]",NaN,11.97864
8984,13448,445,436,48.371650,POINT (-74.01158 40.70319),445,2,"[2, 2, 10, 4]",735.0,1.73736,POINT (-74.01158 40.70319),POINT (-74.01197 40.70254),436,1,"[1, 1, 2, 10]",742.0,3.456432


In [37]:
columns_to_keep = ['edgeID', 'id_x', 'id_y', 'elevation_x', 'elevation_y', 'closest_road_id_x', 'closest_road_id_y','geometry_x', 'geometry_y']
merged_df = merged_df.loc[:, columns_to_keep]

In [38]:
merged_df

,edgeID,id_x,id_y,elevation_x,elevation_y,closest_road_id_x,closest_road_id_y,geometry_x,geometry_y
0,0,293,312,3.023616,3.194304,"[88, 174]",[174],POINT (-74.01267 40.71804),POINT (-74.01311 40.71625)
1,1,312,304,3.194304,3.255264,[174],"[79, 80, 174]",POINT (-74.01311 40.71625),POINT (-74.01294 40.71692)
2,2,293,304,3.023616,3.255264,"[88, 174]","[79, 80, 174]",POINT (-74.01267 40.71804),POINT (-74.01294 40.71692)
3,3,312,307,3.194304,2.740152,[174],[79],POINT (-74.01311 40.71625),POINT (-74.01239 40.71655)
4,4,307,304,2.740152,3.255264,[79],"[79, 80, 174]",POINT (-74.01239 40.71655),POINT (-74.01294 40.71692)
...,...,...,...,...,...,...,...,...,...
8981,13442,883,912,12.429744,11.97864,[80],"[18, 22]",POINT (-74.00483 40.71329),POINT (-74.00448 40.71246)
8982,13445,883,911,12.429744,12.807696,[80],"[18, 22]",POINT (-74.00483 40.71329),POINT (-74.00464 40.71233)
8983,13446,911,912,12.807696,11.97864,"[18, 22]","[18, 22]",POINT (-74.00464 40.71233),POINT (-74.00448 40.71246)
8984,13448,445,436,1.73736,3.456432,"[2, 2, 10, 4]","[1, 1, 2, 10]",POINT (-74.01158 40.70319),POINT (-74.01197 40.70254)


In [39]:
merged_df['geometry'] = merged_df.apply(lambda row: LineString([row['geometry_x'], row['geometry_y']]), axis=1)

In [40]:
merged_df

,edgeID,id_x,id_y,elevation_x,elevation_y,closest_road_id_x,closest_road_id_y,geometry_x,geometry_y,geometry
0,0,293,312,3.023616,3.194304,"[88, 174]",[174],POINT (-74.01267 40.71804),POINT (-74.01311 40.71625),LINESTRING (-74.01266545051853 40.718041514794...
1,1,312,304,3.194304,3.255264,[174],"[79, 80, 174]",POINT (-74.01311 40.71625),POINT (-74.01294 40.71692),LINESTRING (-74.01311010480852 40.716251664262...
2,2,293,304,3.023616,3.255264,"[88, 174]","[79, 80, 174]",POINT (-74.01267 40.71804),POINT (-74.01294 40.71692),LINESTRING (-74.01266545051853 40.718041514794...
3,3,312,307,3.194304,2.740152,[174],[79],POINT (-74.01311 40.71625),POINT (-74.01239 40.71655),LINESTRING (-74.01311010480852 40.716251664262...
4,4,307,304,2.740152,3.255264,[79],"[79, 80, 174]",POINT (-74.01239 40.71655),POINT (-74.01294 40.71692),LINESTRING (-74.01239491348734 40.716551220836...
...,...,...,...,...,...,...,...,...,...,...
8981,13442,883,912,12.429744,11.97864,[80],"[18, 22]",POINT (-74.00483 40.71329),POINT (-74.00448 40.71246),LINESTRING (-74.00483259805303 40.713290359394...
8982,13445,883,911,12.429744,12.807696,[80],"[18, 22]",POINT (-74.00483 40.71329),POINT (-74.00464 40.71233),LINESTRING (-74.00483259805303 40.713290359394...
8983,13446,911,912,12.807696,11.97864,"[18, 22]","[18, 22]",POINT (-74.00464 40.71233),POINT (-74.00448 40.71246),LINESTRING (-74.0046369420439 40.7123298662589...
8984,13448,445,436,1.73736,3.456432,"[2, 2, 10, 4]","[1, 1, 2, 10]",POINT (-74.01158 40.70319),POINT (-74.01197 40.70254),LINESTRING (-74.0115782525511 40.7031941775884...


# Calculate each edge's angle to NORTH

In [41]:
merged_df['angle_to_north'] = merged_df['geometry'].apply(calculate_angle_to_north)

In [42]:
merged_df

,edgeID,id_x,id_y,elevation_x,elevation_y,closest_road_id_x,closest_road_id_y,geometry_x,geometry_y,geometry,angle_to_north
0,0,293,312,3.023616,3.194304,"[88, 174]",[174],POINT (-74.01267 40.71804),POINT (-74.01311 40.71625),LINESTRING (-74.01266545051853 40.718041514794...,-103.951601
1,1,312,304,3.194304,3.255264,[174],"[79, 80, 174]",POINT (-74.01311 40.71625),POINT (-74.01294 40.71692),LINESTRING (-74.01311010480852 40.716251664262...,75.224619
2,2,293,304,3.023616,3.255264,"[88, 174]","[79, 80, 174]",POINT (-74.01267 40.71804),POINT (-74.01294 40.71692),LINESTRING (-74.01266545051853 40.718041514794...,-103.463305
3,3,312,307,3.194304,2.740152,[174],[79],POINT (-74.01311 40.71625),POINT (-74.01239 40.71655),LINESTRING (-74.01311010480852 40.716251664262...,22.726283
4,4,307,304,2.740152,3.255264,[79],"[79, 80, 174]",POINT (-74.01239 40.71655),POINT (-74.01294 40.71692),LINESTRING (-74.01239491348734 40.716551220836...,146.013066
...,...,...,...,...,...,...,...,...,...,...,...
8981,13442,883,912,12.429744,11.97864,[80],"[18, 22]",POINT (-74.00483 40.71329),POINT (-74.00448 40.71246),LINESTRING (-74.00483259805303 40.713290359394...,-67.148052
8982,13445,883,911,12.429744,12.807696,[80],"[18, 22]",POINT (-74.00483 40.71329),POINT (-74.00464 40.71233),LINESTRING (-74.00483259805303 40.713290359394...,-78.486169
8983,13446,911,912,12.807696,11.97864,"[18, 22]","[18, 22]",POINT (-74.00464 40.71233),POINT (-74.00448 40.71246),LINESTRING (-74.0046369420439 40.7123298662589...,40.236358
8984,13448,445,436,1.73736,3.456432,"[2, 2, 10, 4]","[1, 1, 2, 10]",POINT (-74.01158 40.70319),POINT (-74.01197 40.70254),LINESTRING (-74.0115782525511 40.7031941775884...,-121.335266


# Exclude all edges when two points do not have same road id

In [43]:
# Extract the common road IDs
merged_df['common_road_id'] = merged_df.apply(
    lambda row: set(row['closest_road_id_x']).intersection(row['closest_road_id_y']), axis=1
)

# Filter the DataFrame to keep only rows with a common road ID
filtered_edges_df = merged_df[merged_df['common_road_id'].apply(lambda x: len(x) > 0)]

In [44]:
filtered_edges_df

,edgeID,id_x,id_y,elevation_x,elevation_y,closest_road_id_x,closest_road_id_y,geometry_x,geometry_y,geometry,angle_to_north,common_road_id
0,0,293,312,3.023616,3.194304,"[88, 174]",[174],POINT (-74.01267 40.71804),POINT (-74.01311 40.71625),LINESTRING (-74.01266545051853 40.718041514794...,-103.951601,{174}
1,1,312,304,3.194304,3.255264,[174],"[79, 80, 174]",POINT (-74.01311 40.71625),POINT (-74.01294 40.71692),LINESTRING (-74.01311010480852 40.716251664262...,75.224619,{174}
2,2,293,304,3.023616,3.255264,"[88, 174]","[79, 80, 174]",POINT (-74.01267 40.71804),POINT (-74.01294 40.71692),LINESTRING (-74.01266545051853 40.718041514794...,-103.463305,{174}
4,4,307,304,2.740152,3.255264,[79],"[79, 80, 174]",POINT (-74.01239 40.71655),POINT (-74.01294 40.71692),LINESTRING (-74.01239491348734 40.716551220836...,146.013066,{79}
7,8,296,302,2.871216,3.425952,[80],"[80, 189]",POINT (-74.01226 40.71723),POINT (-74.01160 40.71690),LINESTRING (-74.01225730468602 40.717231775929...,-26.565051,{80}
...,...,...,...,...,...,...,...,...,...,...,...,...
8979,13439,880,883,12.89304,12.429744,[80],[80],POINT (-74.00539 40.71355),POINT (-74.00483 40.71329),LINESTRING (-74.00538596858388 40.713547281426...,-24.904769,{80}
8980,13440,883,879,12.429744,10.536936,[80],[80],POINT (-74.00483 40.71329),POINT (-74.00480 40.71371),LINESTRING (-74.00483259805303 40.713290359394...,86.257012,{80}
8983,13446,911,912,12.807696,11.97864,"[18, 22]","[18, 22]",POINT (-74.00464 40.71233),POINT (-74.00448 40.71246),LINESTRING (-74.0046369420439 40.7123298662589...,40.236358,"{18, 22}"
8984,13448,445,436,1.73736,3.456432,"[2, 2, 10, 4]","[1, 1, 2, 10]",POINT (-74.01158 40.70319),POINT (-74.01197 40.70254),LINESTRING (-74.0115782525511 40.7031941775884...,-121.335266,"{2, 10}"


# Version 2: Find the closest road(s) of each start hydrant

In [45]:
from shapely.ops import nearest_points

# Assuming you have your edge and road DataFrames named df_edge and df_road, respectively

# Initialize a list to store the target road ID for each edge
target_road_ids = []

# Iterate through each edge
for _, edge_row in filtered_edges_df.iterrows():
    edge_geometry = edge_row['geometry']
    common_road_ids = edge_row['common_road_id']
    
    # Convert the set to a list
    common_road_ids = list(common_road_ids)

    # If there is only one common road ID, set it as the target road
    if len(common_road_ids) == 1:
        target_road_ids.append(common_road_ids[0])
    else:
        # Initialize variables to track the closest road and its distance
        closest_road_id = None
        closest_distance = float('inf')

        # Iterate through all common road IDs for this edge
        for road_id in common_road_ids:
            # Find the corresponding road geometry
            road_geometry = df_road[df_road['id'] == road_id]['geometry'].iloc[0]

            # Find the two nearest points on the edge and road
            edge_point, road_point = nearest_points(edge_geometry, road_geometry)

            # Calculate the distance between the edge and road points
            distance = edge_point.distance(road_point)

            # Update the closest road and distance if this road is closer
            if distance < closest_distance:
                closest_road_id = road_id
                closest_distance = distance

        # Append the ID of the closest road to the target_road_ids list
        target_road_ids.append(closest_road_id)

# Add the target_road_id column to the edge DataFrame
filtered_edges_df['target_road_id'] = target_road_ids


/var/folders/d2/rwpl453n4zv0prg3wq7dgx900000gn/T/ipykernel_42072/3678211963.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_edges_df['target_road_id'] = target_road_ids


In [46]:
filtered_edges_df

,edgeID,id_x,id_y,elevation_x,elevation_y,closest_road_id_x,closest_road_id_y,geometry_x,geometry_y,geometry,angle_to_north,common_road_id,target_road_id
0,0,293,312,3.023616,3.194304,"[88, 174]",[174],POINT (-74.01267 40.71804),POINT (-74.01311 40.71625),LINESTRING (-74.01266545051853 40.718041514794...,-103.951601,{174},174
1,1,312,304,3.194304,3.255264,[174],"[79, 80, 174]",POINT (-74.01311 40.71625),POINT (-74.01294 40.71692),LINESTRING (-74.01311010480852 40.716251664262...,75.224619,{174},174
2,2,293,304,3.023616,3.255264,"[88, 174]","[79, 80, 174]",POINT (-74.01267 40.71804),POINT (-74.01294 40.71692),LINESTRING (-74.01266545051853 40.718041514794...,-103.463305,{174},174
4,4,307,304,2.740152,3.255264,[79],"[79, 80, 174]",POINT (-74.01239 40.71655),POINT (-74.01294 40.71692),LINESTRING (-74.01239491348734 40.716551220836...,146.013066,{79},79
7,8,296,302,2.871216,3.425952,[80],"[80, 189]",POINT (-74.01226 40.71723),POINT (-74.01160 40.71690),LINESTRING (-74.01225730468602 40.717231775929...,-26.565051,{80},80
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8979,13439,880,883,12.89304,12.429744,[80],[80],POINT (-74.00539 40.71355),POINT (-74.00483 40.71329),LINESTRING (-74.00538596858388 40.713547281426...,-24.904769,{80},80
8980,13440,883,879,12.429744,10.536936,[80],[80],POINT (-74.00483 40.71329),POINT (-74.00480 40.71371),LINESTRING (-74.00483259805303 40.713290359394...,86.257012,{80},80
8983,13446,911,912,12.807696,11.97864,"[18, 22]","[18, 22]",POINT (-74.00464 40.71233),POINT (-74.00448 40.71246),LINESTRING (-74.0046369420439 40.7123298662589...,40.236358,"{18, 22}",22
8984,13448,445,436,1.73736,3.456432,"[2, 2, 10, 4]","[1, 1, 2, 10]",POINT (-74.01158 40.70319),POINT (-74.01197 40.70254),LINESTRING (-74.0115782525511 40.7031941775884...,-121.335266,"{2, 10}",10


In [47]:
road_geometry_dict = dict(zip(df_road['id'], df_road['geometry']))

In [48]:
filtered_edges_df['road_geometry'] = filtered_edges_df['target_road_id'].map(road_geometry_dict)

/var/folders/d2/rwpl453n4zv0prg3wq7dgx900000gn/T/ipykernel_42072/2460844273.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_edges_df['road_geometry'] = filtered_edges_df['target_road_id'].map(road_geometry_dict)


In [49]:
road_north_dict = dict(zip(df_road['id'], df_road['angle_to_north']))

In [50]:
filtered_edges_df['road_angle_to_north'] = filtered_edges_df['target_road_id'].map(road_north_dict)

/var/folders/d2/rwpl453n4zv0prg3wq7dgx900000gn/T/ipykernel_42072/1537361309.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_edges_df['road_angle_to_north'] = filtered_edges_df['target_road_id'].map(road_north_dict)


In [51]:
# Initialize a list to store the target road ID for each edge
angle_differences = []

# Iterate through each edge
for _, edge_row in filtered_edges_df.iterrows():
    edge_angle = edge_row['angle_to_north']
    road_angle = edge_row['road_angle_to_north']
    
    angle_difference = abs(abs(edge_angle) - abs(road_angle))
    
    angle_differences.append(angle_difference)

# Add the target_road_id column to the edge DataFrame
filtered_edges_df['angle_differences'] = angle_differences

/var/folders/d2/rwpl453n4zv0prg3wq7dgx900000gn/T/ipykernel_42072/3945816542.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_edges_df['angle_differences'] = angle_differences


In [61]:
filtered_edges_df['slope_differences'] = filtered_edges_df['elevation_x'] - filtered_edges_df['elevation_y']

/var/folders/d2/rwpl453n4zv0prg3wq7dgx900000gn/T/ipykernel_42072/2130671145.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_edges_df['slope_differences'] = filtered_edges_df['elevation_x'] - filtered_edges_df['elevation_y']


In [62]:
filtered_edges_df

,edgeID,id_x,id_y,elevation_x,elevation_y,closest_road_id_x,closest_road_id_y,geometry_x,geometry_y,geometry,angle_to_north,common_road_id,target_road_id,road_geometry,road_angle_to_north,angle_differences,slope_differences
0,0,293,312,3.023616,3.194304,"[88, 174]",[174],POINT (-74.01267 40.71804),POINT (-74.01311 40.71625),LINESTRING (-74.01266545051853 40.718041514794...,-103.951601,{174},174,LINESTRING (-74.00847236404077 40.742318625728...,-102.724099,1.227502,-0.170688
1,1,312,304,3.194304,3.255264,[174],"[79, 80, 174]",POINT (-74.01311 40.71625),POINT (-74.01294 40.71692),LINESTRING (-74.01311010480852 40.716251664262...,75.224619,{174},174,LINESTRING (-74.00847236404077 40.742318625728...,-102.724099,27.499479,-0.06096
2,2,293,304,3.023616,3.255264,"[88, 174]","[79, 80, 174]",POINT (-74.01267 40.71804),POINT (-74.01294 40.71692),LINESTRING (-74.01266545051853 40.718041514794...,-103.463305,{174},174,LINESTRING (-74.00847236404077 40.742318625728...,-102.724099,0.739206,-0.231648
4,4,307,304,2.740152,3.255264,[79],"[79, 80, 174]",POINT (-74.01239 40.71655),POINT (-74.01294 40.71692),LINESTRING (-74.01239491348734 40.716551220836...,146.013066,{79},79,LINESTRING (-74.01283152205522 40.716579129173...,-26.006766,120.006299,-0.515112
7,8,296,302,2.871216,3.425952,[80],"[80, 189]",POINT (-74.01226 40.71723),POINT (-74.01160 40.71690),LINESTRING (-74.01225730468602 40.717231775929...,-26.565051,{80},80,LINESTRING (-74.01275835971087 40.717237590272...,-27.275880,0.710829,-0.554736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8979,13439,880,883,12.89304,12.429744,[80],[80],POINT (-74.00539 40.71355),POINT (-74.00483 40.71329),LINESTRING (-74.00538596858388 40.713547281426...,-24.904769,{80},80,LINESTRING (-74.01275835971087 40.717237590272...,-27.275880,2.371112,0.463296
8980,13440,883,879,12.429744,10.536936,[80],[80],POINT (-74.00483 40.71329),POINT (-74.00480 40.71371),LINESTRING (-74.00483259805303 40.713290359394...,86.257012,{80},80,LINESTRING (-74.01275835971087 40.717237590272...,-27.275880,58.981131,1.892808
8983,13446,911,912,12.807696,11.97864,"[18, 22]","[18, 22]",POINT (-74.00464 40.71233),POINT (-74.00448 40.71246),LINESTRING (-74.0046369420439 40.7123298662589...,40.236358,"{18, 22}",22,LINESTRING (-74.0031618485947 40.7139743314425...,8.130102,32.106256,0.829056
8984,13448,445,436,1.73736,3.456432,"[2, 2, 10, 4]","[1, 1, 2, 10]",POINT (-74.01158 40.70319),POINT (-74.01197 40.70254),LINESTRING (-74.0115782525511 40.7031941775884...,-121.335266,"{2, 10}",10,LINESTRING (-74.01161150008265 40.701711283402...,70.693821,50.641446,-1.719072


In [65]:
# Create a new DataFrame to store the results
closest_two_roads = []

# Iterate through unique start hydrants (id_x)
for start_hydrant in filtered_edges_df['id_x'].unique():
    # Filter edges that start from the current hydrant
    hydrant_edges = filtered_edges_df[filtered_edges_df['id_x'] == start_hydrant]
    
    # Filter edges with angle difference between 30 and 150
    hydrant_edges = hydrant_edges[(hydrant_edges['angle_differences'] <= 30) | (hydrant_edges['angle_differences'] >= 150)]
    
    # Handle angle differences greater than 150 by subtracting 180 and taking the absolute value
    hydrant_edges['angle_differences'] = hydrant_edges['angle_differences'].apply(lambda x: abs(x - 180) if x > 150 else x)
    
    # Sort the hydrant_edges by angle_difference in ascending order
    hydrant_edges = hydrant_edges.sort_values(by='angle_differences')
    
    # Keep the top 2 edges with the lowest angle difference
    top_2_edges = hydrant_edges.head(2)
    
    # Append the top 2 edges to the closest_two_roads list
    closest_two_roads.append(top_2_edges)

# Concatenate the selected edges for all hydrants
selected_edges_df = pd.concat(closest_two_roads)

In [66]:
selected_edges_df['edge_length'] = selected_edges_df['geometry'].apply(lambda geom: geom.length)

In [67]:
selected_edges_df

,edgeID,id_x,id_y,elevation_x,elevation_y,closest_road_id_x,closest_road_id_y,geometry_x,geometry_y,geometry,angle_to_north,common_road_id,target_road_id,road_geometry,road_angle_to_north,angle_differences,slope_differences,edge_length
2,2,293,304,3.023616,3.255264,"[88, 174]","[79, 80, 174]",POINT (-74.01267 40.71804),POINT (-74.01294 40.71692),LINESTRING (-74.01266545051853 40.718041514794...,-103.463305,{174},174,LINESTRING (-74.00847236404077 40.742318625728...,-102.724099,0.739206,-0.231648,0.001158
0,0,293,312,3.023616,3.194304,"[88, 174]",[174],POINT (-74.01267 40.71804),POINT (-74.01311 40.71625),LINESTRING (-74.01266545051853 40.718041514794...,-103.951601,{174},174,LINESTRING (-74.00847236404077 40.742318625728...,-102.724099,1.227502,-0.170688,0.001844
1,1,312,304,3.194304,3.255264,[174],"[79, 80, 174]",POINT (-74.01311 40.71625),POINT (-74.01294 40.71692),LINESTRING (-74.01311010480852 40.716251664262...,75.224619,{174},174,LINESTRING (-74.00847236404077 40.742318625728...,-102.724099,27.499479,-0.06096,0.000686
1523,1994,307,308,2.740152,4.059936,[79],"[79, 189]",POINT (-74.01239 40.71655),POINT (-74.01178 40.71623),LINESTRING (-74.01239491348734 40.716551220836...,-27.529072,{79},79,LINESTRING (-74.01283152205522 40.716579129173...,-26.006766,1.522305,-1.319784,0.000697
7,8,296,302,2.871216,3.425952,[80],"[80, 189]",POINT (-74.01226 40.71723),POINT (-74.01160 40.71690),LINESTRING (-74.01225730468602 40.717231775929...,-26.565051,{80},80,LINESTRING (-74.01275835971087 40.717237590272...,-27.275880,0.710829,-0.554736,0.000737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8979,13439,880,883,12.89304,12.429744,[80],[80],POINT (-74.00539 40.71355),POINT (-74.00483 40.71329),LINESTRING (-74.00538596858388 40.713547281426...,-24.904769,{80},80,LINESTRING (-74.01275835971087 40.717237590272...,-27.275880,2.371112,0.463296,0.000610
8928,13357,880,879,12.89304,10.536936,[80],[80],POINT (-74.00539 40.71355),POINT (-74.00480 40.71371),LINESTRING (-74.00538596858388 40.713547281426...,15.945396,{80},80,LINESTRING (-74.01275835971087 40.717237590272...,-27.275880,11.330485,2.356104,0.000604
8965,13416,910,913,9.61644,5.65404,"[18, 22, 75, 81]","[9, 81]",POINT (-74.00499 40.71189),POINT (-74.00480 40.71141),LINESTRING (-74.00499268024231 40.711887169834...,-68.362059,{81},81,LINESTRING (-74.00528876011323 40.711882966958...,-39.913090,28.448969,3.9624,0.000515
8970,13424,1057,1058,13.554456,9.012936,"[22, 23, 80, 249]",[22],POINT (-74.00295 40.71240),POINT (-74.00335 40.71232),LINESTRING (-74.00294749494648 40.712395246270...,-170.036196,{22},22,LINESTRING (-74.0031618485947 40.7139743314425...,8.130102,18.093907,4.54152,0.000407


In [56]:
selected_edges_gdf = selected_edges_df.loc[:, ['geometry', 'angle_differences']]

geometry = selected_edges_gdf['geometry'].apply(LineString)

selected_edges_gdf = gpd.GeoDataFrame(selected_edges_gdf, geometry=geometry)

# Define the output shapefile path
output_shapefile = 'same_north.shp'

# Save the GeoDataFrame to a shapefile
selected_edges_gdf.to_file(output_shapefile)

/var/folders/d2/rwpl453n4zv0prg3wq7dgx900000gn/T/ipykernel_42072/3780823607.py:11: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  selected_edges_gdf.to_file(output_shapefile)


# Create new list for GA

In [82]:
edge_list = []

for index, row in selected_edges_df.iterrows():
    edge_id = row['edgeID']
    id_x = row['id_x']
    id_y = row['id_y']
    length = row['edge_length']

    edge = Edge(edge_id, id_x, id_y, length)
    edge.slope_difference = row['slope_differences']
    edge.angle_difference = row['angle_differences']
    edge_list.append(edge)

# Version 2.2: Use cost function on current edge dataframe

In [89]:
# sorry I know this is repetitive but I cannot find any other way at this point
def find_max_min_slope(edges):
    max_slope = float('-inf')
    min_slope = float('inf')

    for edge in edges:
        rise = edge.slope_difference
        run = edge.length
        slope = rise / run

        # Check for maximum slope
        if slope > max_slope:
            max_slope = slope

        # Check for minimum slope
        if slope < min_slope:
            min_slope = slope

    return max_slope, min_slope

In [90]:
def normalize_min_max(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val)

max_edge_length = selected_edges_df['edge_length'].max()
max_slope, min_slope = find_max_min_slope(edge_list)

In [91]:
def costLength(edge):
    length = edge.length
    return normalize_min_max(length, 0, max_edge_length)

def costSlope(edge):
    hydrant_j = df_point.iloc[edge.p1]
    hydrant_i = df_point.iloc[edge.p2]
    rise = hydrant_j.elevation - hydrant_i.elevation
    run = edge.length
    slope = rise/run
    if slope < 0:
        return 1
    elif slope > 0.006146721416: #too steep
      cost = normalize_min_max(50*slope, 0, max_slope)
      if cost >1:
        cost = 1
      return cost
    elif slope > 0.001702866396: # in max range
      cost = normalize_min_max(25*slope, 0, max_slope)
      if cost >1:
        cost = 1
      return cost
    elif slope > 0.000661225445: #just right
      cost = normalize_min_max(slope, 0, max_slope)
      if cost >1:
        cost = 1
      return cost
    else: #too flat
      cost = normalize_min_max(100*slope, 0, max_slope)
      if cost >1:
        cost = 1
      return cost

def costAngle(edge):
    angle_diff = edge.angle_difference
    mean = 90  # Mean angle (centered around 90 degrees)
    std_dev = 20  # Standard deviation controls the spread of the curve

    # Calculate the cost as a Gaussian-like curve
    cost = np.exp(-((angle_diff - mean) ** 2) / (2 * std_dev ** 2))

    # Normalize the cost to a range of [0, 1]
    normalized_cost = 1 - cost  # Invert the curve for higher cost near 90 degrees

    return normalized_cost

# Genetic Algorithm

In [92]:
# Define the cost function using the given formula
def cost_function(αL, αS, α𝛳, edge):
    return αL * costLength(edge) + αS * costSlope(edge) + α𝛳 * costAngle(edge)

In [93]:
# Define the fitness function
def fitness(αL, αS, α𝛳, edges):
    total_cost = sum(cost_function(αL, αS, α𝛳, edge) for edge in edges)
    return total_cost

In [94]:
# Genetic Algorithm function
import random
import numpy as np
from concurrent.futures import ThreadPoolExecutor

def genetic_algorithm(edges, population_size=50, num_generations=100, mutation_rate=0.1):
    num_edges = len(edges)

    # Generate an initial population of random individuals (chromosomes)
    population = np.random.rand(population_size, 3)  # [αL, αS, α𝛳]

    with ThreadPoolExecutor() as executor:
        for generation in range(num_generations):
            # Evaluate the fitness of each individual in the population using parallelism
            fitness_scores = list(executor.map(lambda ind: fitness(ind[0], ind[1], ind[2], edges), population))

            # Perform selection using indices of sorted fitness scores
            selected_indices = np.argsort(fitness_scores)[:population_size // 2]
            selected_population = population[selected_indices]

            # Perform crossover and mutation using NumPy vectorization
            parents = selected_population[np.random.choice(len(selected_population), size=(population_size, 2))]
            children = (parents[:, 0] + parents[:, 1]) / 2

            # Apply mutation using NumPy vectorization
            mask = np.random.rand(*children.shape) < mutation_rate
            mutation = np.random.uniform(-0.1, 0.1, size=children.shape)
            children += mask * mutation

            # Replace old population with new population
            population = children

    # Find the best individual
    best_index = np.argmin(fitness_scores)
    best_individual = {'αL': population[best_index, 0], 'αS': population[best_index, 1], 'α𝛳': population[best_index, 2]}

    return best_individual

In [95]:
best_weights = genetic_algorithm(edge_list)

In [96]:
best_weights

{'αL': 0.21318401369565865,
 'αS': -0.8127984280132056,
 'α𝛳': -0.8591231312224374}

In [98]:
# Assuming you have already found the best weights
best_αL = best_weights['αL']
best_αS = best_weights['αS']
best_α𝛳 = best_weights['α𝛳']

# Calculate the cost for each edge using the best weights
edge_costs = [cost_function(best_αL, best_αS, best_α𝛳, edge) for edge in edge_list]

# Add the calculated costs to the edge list as a new attribute
for edge, cost in zip(edge_list, edge_costs):
    edge.cost = cost

In [99]:
selected_edges_df['cost'] = edge_costs

In [100]:
selected_edges_df

,edgeID,id_x,id_y,elevation_x,elevation_y,closest_road_id_x,closest_road_id_y,geometry_x,geometry_y,geometry,angle_to_north,common_road_id,target_road_id,road_geometry,road_angle_to_north,angle_differences,slope_differences,edge_length,cost
2,2,293,304,3.023616,3.255264,"[88, 174]","[79, 80, 174]",POINT (-74.01267 40.71804),POINT (-74.01294 40.71692),LINESTRING (-74.01266545051853 40.718041514794...,-103.463305,{174},174,LINESTRING (-74.00847236404077 40.742318625728...,-102.724099,0.739206,-0.231648,0.001158,-1.614525
0,0,293,312,3.023616,3.194304,"[88, 174]",[174],POINT (-74.01267 40.71804),POINT (-74.01311 40.71625),LINESTRING (-74.01266545051853 40.718041514794...,-103.951601,{174},174,LINESTRING (-74.00847236404077 40.742318625728...,-102.724099,1.227502,-0.170688,0.001844,-1.580527
1,1,312,304,3.194304,3.255264,[174],"[79, 80, 174]",POINT (-74.01311 40.71625),POINT (-74.01294 40.71692),LINESTRING (-74.01311010480852 40.716251664262...,75.224619,{174},174,LINESTRING (-74.00847236404077 40.742318625728...,-102.724099,27.499479,-0.06096,0.000686,-1.631415
1523,1994,307,308,2.740152,4.059936,[79],"[79, 189]",POINT (-74.01239 40.71655),POINT (-74.01178 40.71623),LINESTRING (-74.01239491348734 40.716551220836...,-27.529072,{79},79,LINESTRING (-74.01283152205522 40.716579129173...,-26.006766,1.522305,-1.319784,0.000697,-1.637363
7,8,296,302,2.871216,3.425952,[80],"[80, 189]",POINT (-74.01226 40.71723),POINT (-74.01160 40.71690),LINESTRING (-74.01225730468602 40.717231775929...,-26.565051,{80},80,LINESTRING (-74.01275835971087 40.717237590272...,-27.275880,0.710829,-0.554736,0.000737,-1.635386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8979,13439,880,883,12.89304,12.429744,[80],[80],POINT (-74.00539 40.71355),POINT (-74.00483 40.71329),LINESTRING (-74.00538596858388 40.713547281426...,-24.904769,{80},80,LINESTRING (-74.01275835971087 40.717237590272...,-27.275880,2.371112,0.463296,0.000610,-1.641644
8928,13357,880,879,12.89304,10.536936,[80],[80],POINT (-74.00539 40.71355),POINT (-74.00480 40.71371),LINESTRING (-74.00538596858388 40.713547281426...,15.945396,{80},80,LINESTRING (-74.01275835971087 40.717237590272...,-27.275880,11.330485,2.356104,0.000604,-1.641615
8965,13416,910,913,9.61644,5.65404,"[18, 22, 75, 81]","[9, 81]",POINT (-74.00499 40.71189),POINT (-74.00480 40.71141),LINESTRING (-74.00499268024231 40.711887169834...,-68.362059,{81},81,LINESTRING (-74.00528876011323 40.711882966958...,-39.913090,28.448969,3.9624,0.000515,-1.638896
8970,13424,1057,1058,13.554456,9.012936,"[22, 23, 80, 249]",[22],POINT (-74.00295 40.71240),POINT (-74.00335 40.71232),LINESTRING (-74.00294749494648 40.712395246270...,-170.036196,{22},22,LINESTRING (-74.0031618485947 40.7139743314425...,8.130102,18.093907,4.54152,0.000407,-1.650434


# Fitering only two each hydrant

In [101]:
sorted_df = selected_edges_df.groupby('id_x').apply(lambda x: x.nsmallest(2, 'cost')).reset_index(drop=True)

In [102]:
sorted_df

,edgeID,id_x,id_y,elevation_x,elevation_y,closest_road_id_x,closest_road_id_y,geometry_x,geometry_y,geometry,angle_to_north,common_road_id,target_road_id,road_geometry,road_angle_to_north,angle_differences,slope_differences,edge_length,cost
0,11915,2,7,2.465832,2.109216,"[120, 170]",[120],POINT (-74.00721 40.74697),POINT (-74.00642 40.74663),LINESTRING (-74.00720977640854 40.746973999302...,-23.657997,{120},120,LINESTRING (-73.99101751982766 40.739705086511...,-23.477400,0.180597,0.356616,0.000858,-1.629363
1,60,6,0,2.292096,1.606296,[119],"[119, 170]",POINT (-74.00646 40.74724),POINT (-74.00698 40.74748),LINESTRING (-74.00645714051556 40.747243600218...,155.618188,{119},119,LINESTRING (-73.99069124542119 40.740191113595...,156.321185,0.702997,0.6858,0.000571,-1.643577
2,11920,7,16,2.109216,2.862072,[120],"[120, 167]",POINT (-74.00642 40.74663),POINT (-74.00561 40.74628),LINESTRING (-74.00642344040097 40.746629509241...,-23.517962,{120},120,LINESTRING (-73.99101751982766 40.739705086511...,-23.477400,0.040562,-0.752856,0.000882,-1.628196
3,11907,7,4,2.109216,2.267712,[120],"[120, 121, 170]",POINT (-74.00642 40.74663),POINT (-74.00728 40.74667),LINESTRING (-74.00642344040097 40.746629509241...,177.261853,{120},120,LINESTRING (-73.99101751982766 40.739705086511...,-23.477400,26.215546,-0.158496,0.000862,-1.623901
4,3718,8,11,1.923288,2.127504,"[122, 170]","[123, 170]",POINT (-74.00799 40.74604),POINT (-74.00809 40.74549),LINESTRING (-74.00799236795895 40.746037885007...,-99.782407,{170},170,LINESTRING (-74.00725292282137 40.747627500928...,-102.474240,2.691833,-0.204216,0.000551,-1.644569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2604,883,2296,2293,6.5532,5.952744,"[30, 35, 235]","[35, 235]",POINT (-73.97987 40.71361),POINT (-73.97965 40.71397),LINESTRING (-73.97987404430187 40.713608009757...,58.535856,"{35, 235}",235,LINESTRING (-73.97989490749605 40.713792293980...,63.176682,4.640826,0.600456,0.000424,-1.650839
2605,881,2296,1944,6.5532,5.355336,"[30, 35, 235]","[30, 35, 236]",POINT (-73.97987 40.71361),POINT (-73.97912 40.71337),LINESTRING (-73.97987404430187 40.713608009757...,-17.516336,"{35, 30}",30,LINESTRING (-73.99658731157727 40.711278672363...,5.115145,12.401191,1.197864,0.000789,-1.632368
2606,2462,2297,2234,7.303008,7.287768,[221],[221],POINT (-73.98387 40.72077),POINT (-73.98388 40.72078),LINESTRING (-73.98386708301707 40.720766081186...,132.273689,{221},221,LINESTRING (-73.9838002800147 40.7213944873012...,-103.480157,28.793532,0.01524,0.000020,-1.662979
2607,2393,2297,2205,7.303008,7.674864,[221],"[199, 221]",POINT (-73.98387 40.72077),POINT (-73.98424 40.71999),LINESTRING (-73.98386708301707 40.720766081186...,-115.758122,{221},221,LINESTRING (-73.9838002800147 40.7213944873012...,-103.480157,12.277965,-0.371856,0.000856,-1.629048


In [103]:
geometry = sorted_df['geometry'].apply(LineString)

In [105]:
sorted_gdf = gpd.GeoDataFrame(sorted_df, geometry=geometry)

In [107]:
sorted_gdf = sorted_gdf.loc[:, ['geometry', 'cost']]

In [108]:
# Define the output shapefile path
output_shapefile = 'Charan_cost_function_two.shp'

# Save the GeoDataFrame to a shapefile
sorted_gdf.to_file(output_shapefile)

# Version 1: Find the closest road of each edge and calculate angle difference

In [45]:
# Create a new DataFrame to store the results
closest_roads = []

# Iterate through each edge
for edge_id, edge_row in edge_gdf.iterrows():
    edge_geometry = edge_row['geometry']
    min_distance = float('inf')
    closest_road = None
    
    # Iterate through each road segment
    for road_id, road_row in road_gdf.iterrows():
        road_geometry = road_row['geometry']
        # Calculate the distance between the edge and the road
        distance = edge_geometry.distance(road_geometry)
        
        # Check if this road is closer than the previous closest road
        if distance < min_distance:
            min_distance = distance
            closest_road = road_row
    
    # Calculate the angle_to_north difference between the edge and the closest road
    angle_difference = abs(abs(edge_row['angle_to_north']) - abs(closest_road['angle_to_north']))
    
    # Store the result for this edge
    closest_roads.append({
        'edgeID': edge_row['edgeID'],
        'closest_road_id': closest_road['id'],
        'min_distance': min_distance,
        'closest_road_geometry': closest_road['geometry'],
        'closest_road_angle_to_north': closest_road['angle_to_north'],
        'angle_difference': angle_difference
    })

# Create a DataFrame from the results
closest_roads_df = pd.DataFrame(closest_roads)

# Merge the closest road information back into edge_df
edge_df = merged_df.merge(closest_roads_df, on='edgeID', how='left')

In [46]:
edge_df

,edgeID,id_x,id_y,closest_road_id_x,closest_road_id_x,geometry_x,geometry_y,geometry,angle_to_north,closest_road_id,min_distance,closest_road_geometry,closest_road_angle_to_north,angle_difference
0,0,293,312,"[88, 174]","[88, 174]",POINT (-74.01267 40.71804),POINT (-74.01311 40.71625),LINESTRING (-74.01266545051853 40.718041514794...,-103.951601,80,0.000104,LINESTRING (-74.01275835971087 40.717237590272...,-27.275880,76.675721
1,1,312,304,[174],[174],POINT (-74.01311 40.71625),POINT (-74.01294 40.71692),LINESTRING (-74.01311010480852 40.716251664262...,75.224619,174,0.000165,LINESTRING (-74.00847236404077 40.742318625728...,-102.724099,27.499479
2,2,293,304,"[88, 174]","[88, 174]",POINT (-74.01267 40.71804),POINT (-74.01294 40.71692),LINESTRING (-74.01266545051853 40.718041514794...,-103.463305,80,0.000097,LINESTRING (-74.01275835971087 40.717237590272...,-27.275880,76.187424
3,3,312,307,[174],[174],POINT (-74.01311 40.71625),POINT (-74.01239 40.71655),LINESTRING (-74.01311010480852 40.716251664262...,22.726283,79,0.000000,LINESTRING (-74.01283152205522 40.716579129173...,-26.006766,3.280483
4,4,307,304,[79],[79],POINT (-74.01239 40.71655),POINT (-74.01294 40.71692),LINESTRING (-74.01239491348734 40.716551220836...,146.013066,174,0.000000,LINESTRING (-74.00847236404077 40.742318625728...,-102.724099,43.288967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8981,13442,883,912,[80],[80],POINT (-74.00483 40.71329),POINT (-74.00448 40.71246),LINESTRING (-74.00483259805303 40.713290359394...,-67.148052,18,0.000000,LINESTRING (-74.0090506412555 40.7089919859351...,40.932708,26.215344
8982,13445,883,911,[80],[80],POINT (-74.00483 40.71329),POINT (-74.00464 40.71233),LINESTRING (-74.00483259805303 40.713290359394...,-78.486169,18,0.000000,LINESTRING (-74.0090506412555 40.7089919859351...,40.932708,37.553460
8983,13446,911,912,"[18, 22]","[18, 22]",POINT (-74.00464 40.71233),POINT (-74.00448 40.71246),LINESTRING (-74.0046369420439 40.7123298662589...,40.236358,22,0.000077,LINESTRING (-74.00862010265011 40.711383275740...,12.287297,27.949061
8984,13448,445,436,"[2, 2, 10, 4]","[2, 2, 10, 4]",POINT (-74.01158 40.70319),POINT (-74.01197 40.70254),LINESTRING (-74.0115782525511 40.7031941775884...,-121.335266,2,0.000000,LINESTRING (-74.0117937812392 40.7029925171459...,12.492476,108.842791


In [47]:
columns_to_keep = ['edgeID', 'id_x', 'id_y', 'geometry_x', 'geometry_y', 'geometry', 'angle_to_north', 'closest_road_id', 'closest_road_geometry', 'closest_road_angle_to_north', 'angle_difference']
edge_df = edge_df.loc[:, columns_to_keep]

In [48]:
edge_df

,edgeID,id_x,id_y,geometry_x,geometry_y,geometry,angle_to_north,closest_road_id,closest_road_geometry,closest_road_angle_to_north,angle_difference
0,0,293,312,POINT (-74.01267 40.71804),POINT (-74.01311 40.71625),LINESTRING (-74.01266545051853 40.718041514794...,-103.951601,80,LINESTRING (-74.01275835971087 40.717237590272...,-27.275880,76.675721
1,1,312,304,POINT (-74.01311 40.71625),POINT (-74.01294 40.71692),LINESTRING (-74.01311010480852 40.716251664262...,75.224619,174,LINESTRING (-74.00847236404077 40.742318625728...,-102.724099,27.499479
2,2,293,304,POINT (-74.01267 40.71804),POINT (-74.01294 40.71692),LINESTRING (-74.01266545051853 40.718041514794...,-103.463305,80,LINESTRING (-74.01275835971087 40.717237590272...,-27.275880,76.187424
3,3,312,307,POINT (-74.01311 40.71625),POINT (-74.01239 40.71655),LINESTRING (-74.01311010480852 40.716251664262...,22.726283,79,LINESTRING (-74.01283152205522 40.716579129173...,-26.006766,3.280483
4,4,307,304,POINT (-74.01239 40.71655),POINT (-74.01294 40.71692),LINESTRING (-74.01239491348734 40.716551220836...,146.013066,174,LINESTRING (-74.00847236404077 40.742318625728...,-102.724099,43.288967
...,...,...,...,...,...,...,...,...,...,...,...
8981,13442,883,912,POINT (-74.00483 40.71329),POINT (-74.00448 40.71246),LINESTRING (-74.00483259805303 40.713290359394...,-67.148052,18,LINESTRING (-74.0090506412555 40.7089919859351...,40.932708,26.215344
8982,13445,883,911,POINT (-74.00483 40.71329),POINT (-74.00464 40.71233),LINESTRING (-74.00483259805303 40.713290359394...,-78.486169,18,LINESTRING (-74.0090506412555 40.7089919859351...,40.932708,37.553460
8983,13446,911,912,POINT (-74.00464 40.71233),POINT (-74.00448 40.71246),LINESTRING (-74.0046369420439 40.7123298662589...,40.236358,22,LINESTRING (-74.00862010265011 40.711383275740...,12.287297,27.949061
8984,13448,445,436,POINT (-74.01158 40.70319),POINT (-74.01197 40.70254),LINESTRING (-74.0115782525511 40.7031941775884...,-121.335266,2,LINESTRING (-74.0117937812392 40.7029925171459...,12.492476,108.842791


# Failed: two lowest cost

In [116]:
# Create a new DataFrame to store the results
closest_two_roads = []

# Iterate through unique start hydrants (id_x)
for start_hydrant in edge_df['id_x'].unique():
    # Filter edges that start from the current hydrant
    hydrant_edges = edge_df[edge_df['id_x'] == start_hydrant]
    
    # Filter edges with angle difference between 30 and 150
    hydrant_edges = hydrant_edges[(hydrant_edges['angle_difference'] <= 30) | (hydrant_edges['angle_difference'] >= 150)]
    
    # Handle angle differences greater than 150 by subtracting 180 and taking the absolute value
    hydrant_edges['angle_difference'] = hydrant_edges['angle_difference'].apply(lambda x: abs(x - 180) if x > 150 else x)
    
    # Sort the hydrant_edges by angle_difference in ascending order
    hydrant_edges = hydrant_edges.sort_values(by='angle_difference')
    
    # Keep the top 2 edges with the lowest angle difference
    top_2_edges = hydrant_edges.head(2)
    
    # Append the top 2 edges to the closest_two_roads list
    closest_two_roads.append(top_2_edges)

# Concatenate the selected edges for all hydrants
selected_edges_df = pd.concat(closest_two_roads)

In [117]:
selected_edges_df

,edgeID,id_x,id_y,geometry_x,geometry_y,geometry,angle_to_north,closest_road_id,min_distance,closest_road_geometry,closest_road_angle_to_north,angle_difference
3,3,312,307,POINT (-74.01311 40.71625),POINT (-74.01239 40.71655),LINESTRING (-74.01311010480852 40.716251664262...,22.726283,79,0.000000,LINESTRING (-74.01283152205522 40.716579129173...,-26.006766,3.280483
1,1,312,304,POINT (-74.01311 40.71625),POINT (-74.01294 40.71692),LINESTRING (-74.01311010480852 40.716251664262...,75.224619,174,0.000165,LINESTRING (-74.00847236404077 40.742318625728...,-102.724099,27.499479
1523,1994,307,308,POINT (-74.01239 40.71655),POINT (-74.01178 40.71623),LINESTRING (-74.01239491348734 40.716551220836...,-27.529072,79,0.000144,LINESTRING (-74.01283152205522 40.716579129173...,-26.006766,1.522305
6,7,307,302,POINT (-74.01239 40.71655),POINT (-74.01160 40.71690),LINESTRING (-74.01239491348734 40.716551220836...,23.781075,80,0.000000,LINESTRING (-74.01275835971087 40.717237590272...,-27.275880,3.494806
13,16,296,293,POINT (-74.01226 40.71723),POINT (-74.01267 40.71804),LINESTRING (-74.01225730468602 40.717231775929...,116.750217,174,0.000000,LINESTRING (-74.00847236404077 40.742318625728...,-102.724099,14.026118
...,...,...,...,...,...,...,...,...,...,...,...,...
8957,13404,911,1075,POINT (-74.00464 40.71233),POINT (-74.00375 40.71157),LINESTRING (-74.0046369420439 40.7123298662589...,-40.633104,9,0.000000,LINESTRING (-74.0102771901233 40.7050942544037...,38.431706,2.201398
8958,13406,911,1076,POINT (-74.00464 40.71233),POINT (-74.00415 40.71137),LINESTRING (-74.0046369420439 40.7123298662589...,-63.022170,9,0.000000,LINESTRING (-74.0102771901233 40.7050942544037...,38.431706,24.590464
8970,13424,1057,1058,POINT (-74.00295 40.71240),POINT (-74.00335 40.71232),LINESTRING (-74.00294749494648 40.712395246270...,-170.036196,22,0.000202,LINESTRING (-74.00862010265011 40.711383275740...,12.287297,22.251102
8978,13437,883,882,POINT (-74.00483 40.71329),POINT (-74.00441 40.71309),LINESTRING (-74.00483259805303 40.713290359394...,-25.277722,80,0.000180,LINESTRING (-74.01275835971087 40.717237590272...,-27.275880,1.998158


In [118]:
selected_edges_gdf = selected_edges_df.loc[:, ['geometry', 'angle_to_north', 'closest_road_angle_to_north', 'angle_difference']]

In [119]:
geometry = selected_edges_gdf['geometry'].apply(LineString)

In [120]:
selected_edges_gdf = gpd.GeoDataFrame(selected_edges_gdf, geometry=geometry)

In [121]:
# Define the output shapefile path
output_shapefile = 'same_north.shp'

# Save the GeoDataFrame to a shapefile
selected_edges_gdf.to_file(output_shapefile)

/var/folders/d2/rwpl453n4zv0prg3wq7dgx900000gn/T/ipykernel_30656/589445303.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  selected_edges_gdf.to_file(output_shapefile)
